In [1]:
import pandas as pd

In [2]:
concentration = pd.read_csv('../../data/2019/19_concentrations.csv')
labels = pd.read_csv('../../data/2019/19_yellow_labels.csv')
pit = pd.read_csv('../../data/2019/19_time_since_pit.csv')
weather = pd.read_csv('../../data/weather_data/2019_weather.csv', sep=';')
speed = pd.read_csv('../../data/2020/20_avg_speed.csv')

In [3]:
max_concentration = pd.DataFrame(columns=['Time', 'Max Concentration'])
temp_max = 0
for index, row in concentration.iterrows():
    sector = row[1]
    if sector == 1:
        temp_max = 0
    if row[2] > temp_max:
        temp_max = row[2]
    if sector == 13:
        max_concentration.loc[index] = [row[0], temp_max]

In [4]:
start_time = weather['TIME_UTC_SECONDS'].iloc[0]

with open('../../data/2019/19_weather_mod.csv', 'w') as f:
    f.write('Time,Air Temp,Track Temp,Wind Speed\n')
    for index, row in weather.iterrows():
        if 65299 < ((row[0]-start_time)//20 * 20) < 65461:
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time)//20 * 20, 55.5,row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+20)//20 * 20,55.5,row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+40)//20 * 20,55.5,row[3], row[6]))
        else:
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time)//20 * 20, row[2],row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+20)//20 * 20,row[2],row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+40)//20 * 20,row[2],row[3], row[6]))
weather_mod = pd.read_csv('../../data/2019/19_weather_mod.csv')

In [5]:
pit = pit[pit['Last Pit'] < 140]
agg = max_concentration.merge(pit, how='inner', on='Time')
agg = agg.merge(weather_mod, how='inner', on='Time')
# agg = agg.merge(speed, how='inner', on='Time')
agg = agg.merge(labels, how='inner', on='Time')

In [6]:
agg.to_csv('../../data/2019/19_agg_data.csv', index=False)